In [1]:
%matplotlib inline

In [2]:
    from ssf_code import *

This part of the project uses the **sounddevice** module.
Please use ```conda install -c conda-forge python-sounddevice``` or ```pip install sounddevice``` to install the module.
More info about the module: https://python-sounddevice.readthedocs.io/en/0.3.15/installation.html


In [3]:
# This should output your audio devices
sd.query_devices()

> 0 Built-in Microphone, Core Audio (2 in, 0 out)
< 1 Built-in Output, Core Audio (0 in, 2 out)

    * Back to [Introduction](./01_Signals_Sound_and_Filters_Intro.ipynb)

### ACT I, Chapter III
# 3.1 GENERATING SOUND IN PYTHON

And now to generate a sinusoidal wave with a given frequency and sample rate we'll be using:
$$ y = A \sin (\omega t + \phi) = A \sin(2\pi f t + \phi )$$
Where:
- $A$ is amplitude
- $\omega$ is the angular frequency, which can be represented as $2\pi f$(where $f$ is the signal frequency);
- $\phi$ is phase, and we will ignore it for now

Remember the formula for the sinusoidal signal from Chapter 1.2.1? That one: 
$$f(t):=A\sin (2\pi(\omega t - \varphi)), t\in\mathbb{R}$$

Well, the discrepancy comes because we're mixing notions from physics, engineering(signal processing) and math(and programming). And every side has it's own spin on the notation and formulas. In general $\omega$ is used to denote the *angular frequency*(measured in radians per second), which is equal to **"$2\pi$ times the ordinary frequency"**. The ordinary frequency is often denoted by $\nu$ and in the world of programming it is often denoted by $f$ for "convenience"... and is measured in Herz... And lets skip the $\phi$ and $\varphi$ usage. Rant over, let's continue.

In [ ]:
sample_rate = 8000
frequency = 440
duration = .10
T = 1/sample_rate # time duration of 1 sample; we need this to generate the "duration timeline"
N = sample_rate * duration # data points per lenght;
AMPL = 1 # the amplitude
omega = 2*pi*frequency
timeline = np.arange(0, duration, T)
freq_datapoints = np.sin(omega*timeline)*AMPL
plt.plot(timeline, freq_datapoints)
plt.grid(False)
decorate(title="Sinusoidal audio waveform", xlabel="Time(seconds)", ylabel="Amplitude")
plt.show()
print(freq_datapoints)

So far, so good. All of the above code is wrapped in the function ```make_wave``` in the accompanying ```ssf_code.py```
Now let's make a new wave and this time we'll get to see and hear the result.

In [ ]:
sample_rate = 3000
frequency = 6000
duration = .15
sin_wave_data = make_wave(frequency, sample_rate, duration)
plot_play(sin_wave_data[0], sin_wave_data[1], sample_rate)
widget = Audio(sin_wave_data[0], rate=sample_rate)
widget

Oh, no! Doesn't sound and look like we expected. And that is because we tried to generate a pure tone with frequency that is above the sampling rate. That's why we need antialiasing, or we have to chose these values for $f$, that are two times below the sampling rate value.

In [ ]:
sample_rate = 22050
frequency = 6000
duration = .5
amplitude = .2 # to make it not so loud
sin_wave_data = make_wave(frequency, sample_rate, duration, ampl=amplitude)
plot_play(sin_wave_data[0], sin_wave_data[1], sample_rate, ampl=amplitude)
widget = Audio(sin_wave_data[0], rate=sample_rate)
widget

## 3.2 Now let's manipulate the sound in some basic ways

First, let's make a pure-tone 1kHz wave, with sampling rate 8kHz and .2sec duration. Notice the poping sound at the end of the playback.

In [ ]:
f1 = make_wave(1000, 8000, .2)
plot_play(f1[0], f1[1], sample_rate)
widget1 = Audio(f1[0], rate=sample_rate)
widget1

In order to make it sound more smoothly and to avoid the abrupt end, we'll apply some linear fades.

In [ ]:
f2 = fades(f1, 8000, denom=15)
plot_play(f2[0], f2[1], sample_rate)
# This is almost what is happening behind the scenes: 
# envelope1 = np.hanning(len(t_seq))
# f2 = f1 * envelope1
# The fades() function does not use a Hanning curve, but a linear one
widget2 = Audio(f2[0], rate=sample_rate)
widget2

And now lets use all of the above to make a compound wave:

In [ ]:
fund = 440
sr = 22050
dur = .05
ampli = .85
frequencies = [440, 880, 1760, 3520]
print(frequencies)
sub = make_wave(275, sr, dur, ampl=.05)
fc1 = make_wave(fund, sr, dur, ampli)
fc1[0] += sub[0]

for i in range(len(frequencies)):
    f = make_wave(frequencies[i], sr, dur, ampl=ampli-(i/(i+1)))
    fc1[0] += f[0]
                  
fc1 = fades(fc1, sr)
plot_play(fc1[0], fc1[1], sr)
widget = Audio(fc1[0], rate=sr*.5045606740)
widget

# 3.3 LOADING AUDIO FILES
## USING THINKDSP MODULE

All is fine and dandy with the generated signals, but we'll need a real-world examples for the next chapter.
Going further in the project I'll make use of the wonderful ThinkDSP module by Allen Downing.

In [ ]:
guitar_s1 = read_wave('./_audio/guitar_sample.wav')
# wave.normalize()
guitar_s1.plot()
guitar_s1.make_audio()
print(guitar_s1)

The next example computes the spectrum of the audio sample. Comming in next, we'll explore that exact topic. The best [is yet to come!](./05_Signals_Sound_and_Filters_A2C1.ipynb)

In [ ]:
spec_guitar = guitar_s1.make_spectrum()
spec_guitar.plot(high=1000)
decorate(title="A wild spectrogram appears")